In [1]:
# 텐서보드를 이용하기 위해 각종 변수들을 설정하는 방법
import tensorflow as tf
import numpy as np

In [2]:
data = np.loadtxt('./data.csv', delimiter=',',
                  unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])


In [3]:
#######
# 신경망 모델 구성
#######
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# with tf.name_scope 로 묶은 블럭은 텐서보드에서 한 레이어에 표현
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2,10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X,W1))
    
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10,20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))
    
with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20,3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)
    
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)
    
    # tf.summary.scalar를 이용해 수집하고 싶은 값들을 지정 가능
    tf.summary.scalar('cost', cost)

In [6]:
#######
# 신경망 모델 학습
#######
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    print("reload train")
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    print("initialize train")
    sess.run(tf.global_variables_initializer())
    

# 텐서보드에서 표시해주기 위한 텐서들을 수집
merged = tf.summary.merge_all()
# 저장할 그래프와 텐서값들을 저장할 디렉토리를 설정
writer = tf.summary.FileWriter('./logs', sess.graph)
# 저장한 로그는 학습 후 다음의 명령어를 통해 웹서버 실행 뒤 텐서보드에서 확인 가능
# tensorboard --logdir = ./logs
# http://localhost:6666

# 최적화 진행
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    # 적절한 시점에 저장할 값들을 수집하고 저장합니다.
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))

saver.save(sess, './model/dnn.ckpt', global_step=global_step)

#######
# 결과확인
# 0 : 기타, 1: 포유류, 2: 조류
#######
prediction = tf.argmax(model, 1)
target = tf.argmax(Y,1)
print('예측값 : ', sess.run(prediction, feed_dict={X: x_data}))
print('실제값 : ', sess.run(target, feed_dict={Y: y_data}))

check_prediction = tf.equal(prediction, target)
accuracy=tf.reduce_mean(tf.cast(check_prediction, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

initialize train
Step: 1,  Cost: 3.306
Step: 2,  Cost: 3.115
Step: 3,  Cost: 2.935
Step: 4,  Cost: 2.769
Step: 5,  Cost: 2.612
Step: 6,  Cost: 2.464
Step: 7,  Cost: 2.321
Step: 8,  Cost: 2.184
Step: 9,  Cost: 2.053
Step: 10,  Cost: 1.930
Step: 11,  Cost: 1.812
Step: 12,  Cost: 1.696
Step: 13,  Cost: 1.585
Step: 14,  Cost: 1.482
Step: 15,  Cost: 1.386
Step: 16,  Cost: 1.295
Step: 17,  Cost: 1.208
Step: 18,  Cost: 1.129
Step: 19,  Cost: 1.058
Step: 20,  Cost: 0.996
Step: 21,  Cost: 0.942
Step: 22,  Cost: 0.898
Step: 23,  Cost: 0.866
Step: 24,  Cost: 0.840
Step: 25,  Cost: 0.821
Step: 26,  Cost: 0.804
Step: 27,  Cost: 0.788
Step: 28,  Cost: 0.771
Step: 29,  Cost: 0.755
Step: 30,  Cost: 0.739
Step: 31,  Cost: 0.723
Step: 32,  Cost: 0.707
Step: 33,  Cost: 0.692
Step: 34,  Cost: 0.677
Step: 35,  Cost: 0.662
Step: 36,  Cost: 0.649
Step: 37,  Cost: 0.638
Step: 38,  Cost: 0.628
Step: 39,  Cost: 0.619
Step: 40,  Cost: 0.611
Step: 41,  Cost: 0.604
Step: 42,  Cost: 0.599
Step: 43,  Cost: 0.594
Ste